In [1]:
import pandas as pd 
import numpy as np

In [2]:
def read_goog_sp500_dataframe():
  """Returns a dataframe with the results for Google and S&P 500"""
  
  # Point to where you've stored the CSV file on your local machine
  googFile = 'data/GOOG.csv'
  spFile = 'data/SP_500.csv'

  goog = pd.read_csv(googFile, sep=",", usecols=[0,5], names=['Date','Goog'], header=0)
  sp = pd.read_csv(spFile, sep=",", usecols=[0,5], names=['Date','SP500'], header=0)

  goog['SP500'] = sp['SP500']

  # The date object is a string, format it as a date
  goog['Date'] = pd.to_datetime(goog['Date'], format='%Y-%m-%d')

  goog = goog.sort_values(['Date'], ascending=[True])

  returns = goog[[key for key in dict(goog.dtypes) if dict(goog.dtypes)[key] in ['float64', 'int64']]]\
            .pct_change()

  return returns

In [3]:
def read_goog_sp500_logistic_data():
  """Returns a dataframe with the results for Google and 
  S&P 500 set up for logistic regression"""
  returns = read_goog_sp500_dataframe()

  returns['Intercept'] = 1

  # Leave out the first row since it will not have a prediction for UP/DOWN
  # Leave out the last row as it will not have a value for returns
  # Resultant dataframe with the S&P500 and intercept values of all 1s
  xData = np.array(returns[["SP500", "Intercept"]][1:-1])

  yData = (returns["Goog"] > 0)[1:-1]

  return (xData, yData)

In [4]:
def read_goog_sp500_data():
  """Returns a tuple with 2 fields, the returns for Google and the S&P 500.
  Each of the returns are in the form of a 1D array"""

  returns = read_goog_sp500_dataframe()

  # Filter out the very first row which does not have any value for returns
  xData = np.array(returns["SP500"])[1:]
  yData = np.array(returns["Goog"])[1:]

  return (xData, yData)

In [11]:
def read_xom_oil_nasdaq_data():
  """Returns a tuple with 3 fields, the returns for Exxon Mobil, Nasdaq and oil prices.
  Each of the returns are in the form of a 1D array"""

  def readFile(filename):
    # Only read in the date and price at columns 0 and 5
    data = pd.read_csv(filename, sep=",", usecols=[0, 5], names=['Date', 'Price'], header=0)

    # Sort the data in ascending order of date so returns can be calculated
    data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')

    data = data.sort_values(['Date'], ascending=[True])

    # Exclude the date from the percentage change calculation
    returns = data[[key for key in dict(data.dtypes) if dict(data.dtypes)[key] in ['float64', 'int64']]]\
              .pct_change()

    # Filter out the very first row which has no returns associated with it
    return np.array(returns["Price"])[2:]

  nasdaqData = readFile('data/NASDAQ.csv')
  oilData = readFile('data/USO.csv')
  xomData = readFile('data/XOM.csv')

  return (nasdaqData, oilData, xomData)

In [12]:
import numpy as np
from sklearn import datasets, linear_model

In [13]:
nasdaqData, oilData, xomData = read_xom_oil_nasdaq_data()

In [15]:
combined = np.vstack((nasdaqData , oilData)).T

[[1,2,3],[4,5,6]] -> [[1,4],[2,5],[3,6]]

In [19]:
xomNasdaqOilModel = linear_model.LinearRegression()

In [20]:
xomNasdaqOilModel.fit(combined, xomData)
xomNasdaqOilModel.score(combined, xomData)

0.23141361518987388

In [21]:
print xomNasdaqOilModel.coef_
print xomNasdaqOilModel.intercept_

[0.25176347 0.14783892]
0.0029041590931545604


In [22]:
import tensorflow as tf

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [24]:
# Model linear regression y = W1x1 + W2x2 + b
nasdaq_W = tf.Variable(tf.zeros([1, 1]), name="nasdaq_W")
oil_W = tf.Variable(tf.zeros([1, 1]), name="oil_W")

In [25]:
b = tf.Variable(tf.zeros([1]), name="b")

In [26]:
nasdaq_x = tf.placeholder(tf.float32, [None, 1], name="nasdaq_x")
oil_x = tf.placeholder(tf.float32, [None, 1], name="oil_x")

In [27]:
nasdaq_Wx = tf.matmul(nasdaq_x, nasdaq_W)
oil_Wx = tf.matmul(oil_x, oil_W)

In [28]:
y = nasdaq_Wx + oil_Wx + b

In [29]:
y_ = tf.placeholder(tf.float32, [None, 1])

In [30]:
cost = tf.reduce_mean(tf.square(y_ - y))

In [31]:
train_step_ftrl = tf.train.FtrlOptimizer(1).minimize(cost)

In [32]:
all_x_nasdaq = nasdaqData.reshape(-1, 1)
all_x_oil = oilData.reshape(-1, 1)
all_ys = xomData.reshape(-1, 1)

In [33]:
dataset_size = len(oilData)

In [35]:
def trainWithMultiplePointsPerEpoch(steps, train_step, batch_size):

  init = tf.global_variables_initializer()

  with tf.Session() as sess:
    sess.run(init)

    for i in range(steps):

      if dataset_size == batch_size:
        batch_start_idx = 0
      elif dataset_size < batch_size:
        raise ValueError("dataset_size: %d, must be greater than batch_size: %d" % (dataset_size, batch_size))
      else:
        batch_start_idx = (i * batch_size) % dataset_size

      batch_end_idx = batch_start_idx + batch_size

      batch_x_nasdaq = all_x_nasdaq[batch_start_idx : batch_end_idx]
      batch_x_oil = all_x_oil[batch_start_idx : batch_end_idx]
      batch_ys = all_ys[batch_start_idx : batch_end_idx]

      feed = { nasdaq_x: batch_x_nasdaq, oil_x: batch_x_oil, y_: batch_ys }

      sess.run(train_step_ftrl, feed_dict=feed)

      # Print result to screen for every 500 iterations
      if (i + 1) % 500 == 0:
        print("After %d iteration:" % i)
        print("W1: %s" % sess.run(nasdaq_W))
        print("W2: %s" % sess.run(oil_W))
        print("b: %f" % sess.run(b))

        print("cost: %f" % sess.run(cost, feed_dict=feed))

In [36]:
trainWithMultiplePointsPerEpoch(5000, train_step_ftrl, len(oilData))

After 499 iteration:
W1: [[0.25132117]]
W2: [[0.14797106]]
b: 0.002909
cost: 0.001683
After 999 iteration:
W1: [[0.25176248]]
W2: [[0.14783925]]
b: 0.002904
cost: 0.001683
After 1499 iteration:
W1: [[0.25176248]]
W2: [[0.14783925]]
b: 0.002904
cost: 0.001683
After 1999 iteration:
W1: [[0.25176248]]
W2: [[0.14783925]]
b: 0.002904
cost: 0.001683
After 2499 iteration:
W1: [[0.25176248]]
W2: [[0.14783925]]
b: 0.002904
cost: 0.001683
After 2999 iteration:
W1: [[0.25176248]]
W2: [[0.14783925]]
b: 0.002904
cost: 0.001683
After 3499 iteration:
W1: [[0.25176248]]
W2: [[0.14783925]]
b: 0.002904
cost: 0.001683
After 3999 iteration:
W1: [[0.25176248]]
W2: [[0.14783925]]
b: 0.002904
cost: 0.001683
After 4499 iteration:
W1: [[0.25176248]]
W2: [[0.14783925]]
b: 0.002904
cost: 0.001683
After 4999 iteration:
W1: [[0.25176248]]
W2: [[0.14783925]]
b: 0.002904
cost: 0.001683
